# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [ ]:
#Подключение библиотек
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import r2_score

In [ ]:
#Загрузка данных
data = pd.read_csv('/datasets/insurance.csv')

In [ ]:
data

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [ ]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Данные загружены. Пропуски отсутствуют. 
Тип данных столбцов "Возраст" и "Зарплата" можно изменить.

In [ ]:
data['Возраст'] = pd.to_numeric(data['Возраст'])
data['Зарплата'] = pd.to_numeric(data['Зарплата'])

## Умножение матриц

In [ ]:
#Признаки
features = data.drop('Страховые выплаты', axis=1)

#Целевой признак
target = data['Страховые выплаты']

#Матрица признаков (нулевой столбец состоит из единиц)
X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)

#Вектор целевого признака
y = target

#Вектор весов линейной регрессии (нулевой элемент равен сдвигу)
w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
w

array([-9.38235504e-01,  7.92580543e-03,  3.57083050e-02, -1.70080492e-07,
       -1.35676623e-02])

In [ ]:
#Проверка Линейной регрессии и полученной формулы
model = LinearRegression()
model.fit(features, target)
model.coef_

array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

Заменим матрицу $X$ на матрицу $Z$, где $P$ - обратимая матрица с некими значениями, на которую может быть умножена матрица $X$:
$$
Z = XP
$$

Тогда $w1$ будет равно:

$$
w1 = (Z^T Z)^{-1} Z^T y = ((XP)^T XP)^{-1} (XP)^T y 
$$

так как $(AB)^T=B^T A^T$ ,то:

$$
w1 = (P^T X^T  XP)^{-1} P^T X^T  y = (P^T (X^T X) P)^{-1} P^T X^T
$$

так как $(AB)^{-1} = B^{-1} A^{-1}$ ,то:

$$
w1 = P^{-1} (X^T X)^{-1} (P^T)^{-1}  P^T X^T  y
$$

так как $A A^{-1} = A^{-1} A = E$ ,то:

$$
w1 = P^{-1} (X^T X)^{-1} E X^T  y = P^{-1} (X^T X)^{-1} X^T  y
$$


так как $w = (X^T X)^{-1}X^Ty$ ,то:

$$
w1 = P^{-1}w
$$


Так как 
$$
a = Xw
$$

То:
$$
a1 = Zw1 = ZP^{-1} w = XPP^{-1} w = XEw = Xw = a
$$

## Алгоритм преобразования

**Алгоритм**

1. Создание обратимой матрицы со случайными величинами размера исходной матрицы.
2. Умножение исходной матрицы признаков на полученную обратимую матрицу.


**Обоснование**

Создаваемая матрица должна обратимой, чтобы была возможность провести обратное преобразование и получить исходную матрицу.

Создаваемая матрица должна иметь необходимую размерность $(nxn)$, где n - количество признаков для регрессии.
Таким образом матрица, полученная после умножения, будет иметь туже размерность, что и исходная.


## Проверка алгоритма

Проведем исследование модели по двум направлениям:
1. Исследуем качество модели без преобразования:
* С исходными признаками.
* С отмасштабированными признаками.

2. Исследуем качество модели с преобразованием:
* С исходными признаками.
* С отмасштабированными признаками.

In [ ]:
#Разделение данных на обучающую и тестовую выборку
features_train, features_test, target_train, target_test = train_test_split(
    features, 
    target, 
    test_size=0.25, 
    random_state=12345)

In [ ]:
#Модель Линейной регресси на исходных данных
model_1 = LinearRegression()
model_1.fit(features_train, target_train)
r2_1 = r2_score(target_test, model_1.predict(features_test))
print("R2 =", r2_1)


R2 = 0.43522757127026546


In [ ]:
#Модель Линейной регресси на стандартизированных данных
model_2 = LinearRegression()
scaler = StandardScaler()
scaler.fit(features_train)
features_train_scale = scaler.transform(features_train)
features_test_scale = scaler.transform(features_test)
model_2.fit(features_train_scale, target_train)
r2_2 = r2_score(target_test, model_2.predict(features_test_scale))
print("R2 =", r2_2)

R2 = 0.4352275712702668


Результаты моделей идентичны.

In [ ]:
def convert(features):
    convert_features = features
    n = features.shape[1]
    np.random.seed(12345)
    matrix = np.random.randint(1, 10, (n,n))
    det = np.linalg.det(matrix)
    while det == 0:
        np.random.seed(12345)
        matrix = np.random.randint(1, 10, (n,n))
        det = np.linalg.det(matrix)
    convert_features = convert_features @ matrix
    return convert_features, matrix

In [ ]:
display(features.head())
convert_features, matrix = convert(features)
display(convert_features.head())
matrix

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


,0,1,2,3
0,99452.0,396931.0,347287.0,49899.0
1,76279.0,304140.0,266095.0,38329.0
2,42174.0,168087.0,147058.0,21203.0
3,83532.0,333667.0,291948.0,41861.0
4,52371.0,208890.0,182758.0,26301.0


array([[3, 6, 2, 5],
       [6, 3, 2, 7],
       [2, 8, 7, 1],
       [3, 2, 3, 7]])

In [ ]:
#Разделение данных на обучающую и тестовую выборку
features_train, features_test, target_train, target_test = train_test_split(
    features, 
    target, 
    test_size=0.25, 
    random_state=12345)

In [ ]:
#Модель Линейной регресси на преобразованных данных
model_1_convert = LinearRegression()
model_1_convert.fit(features_train, target_train)
r2_1_convert = r2_score(target_test, model_1_convert.predict(features_test))
print("R2 =", r2_1_convert)

R2 = 0.43522757127026546


In [ ]:
#Модель Линейной регресси на стандартизированных преобразованных данных
model_2_convert = LinearRegression()
scaler = StandardScaler()
scaler.fit(features_train)
features_train_scale = scaler.transform(features_train)
features_test_scale = scaler.transform(features_test)
model_2_convert.fit(features_train_scale, target_train)
r2_2_convert = r2_score(target_test, model_2_convert.predict(features_test_scale))
print("R2 =", r2_2_convert)

R2 = 0.4352275712702668


Качество моделей не изменилось.

In [ ]:
#Восстановление данных путем умножения на обратную матрицу
unconvert_features = convert_features.dot(np.linalg.inv(matrix))
display(unconvert_features.head())

,0,1,2,3
0,1.000000e+00,41.0,49600.0,1.000000e+00
1,-8.661377e-12,46.0,38000.0,1.000000e+00
2,-1.381367e-12,29.0,21000.0,-1.073114e-12
3,-7.930517e-12,21.0,41700.0,2.000000e+00
4,1.000000e+00,28.0,26100.0,-2.076811e-12


Данные восстановлены. Имеются небольшие отклонения, корректирующиеся округлением.